In [20]:
# Import necessary libraries

# Data manipulation
import pandas as pd
import numpy as np

# Statistical functions
from scipy.stats import zscore

# For concurrency (running functions in parallel)
from concurrent.futures import ThreadPoolExecutor

# For caching (to speed up repeated function calls)
from functools import lru_cache

# For progress tracking
from tqdm import tqdm

# Plotting and Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Language Detection packages
# `langdetect` for detecting language
from langdetect import detect as langdetect_detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
# `langid` for an alternative language detection method
from langid import classify as langid_classify

# Text Preprocessing and NLP
# Stopwords (common words to ignore) from NLTK
from nltk.corpus import stopwords
# Tokenizing sentences/words
from nltk.tokenize import word_tokenize
# Part-of-speech tagging
from nltk import pos_tag
# Lemmatization (converting words to their base form)
from nltk.stem import WordNetLemmatizer
import nltk
# Regular expressions for text pattern matching
import re

# Word Cloud generation
from wordcloud import WordCloud

In [21]:
import os
import sys
from pathlib import Path

if "workding_dir" not in locals():
    workding_dir = str(Path.cwd().parent)
os.chdir(workding_dir)
sys.path.append(workding_dir)
print("working dir:", workding_dir)

working dir: /home/dariusng2103/projects/dm_project/DM-Fake-News-Detection


In [22]:
from datasets import load_dataset, concatenate_datasets, Dataset

datasets = load_dataset(
    "csv",
    data_files={
        "train": [
            "dataset/train_data_1.csv",
            "dataset/train_data_2.csv",
            "dataset/train_data_3.csv",
            "dataset/train_data_4.csv",
        ],
        "test": "dataset/test_data.csv",
        "rewritten_train": [
            "dataset/rewritten_train_data_1.csv",
            "dataset/rewritten_train_data_2.csv",
            "dataset/rewritten_train_data_3.csv",
            "dataset/rewritten_train_data_4.csv",
        ],
        "rewritten_test": "dataset/rewritten_test_data.csv",
    },
)
datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 54441
    })
    test: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 6050
    })
    rewritten_train: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 54441
    })
    rewritten_test: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 6050
    })
})

In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

def create_basic_nn(vocab_size=10000, embedding_dim=128, input_length=300, dense_units=64, dropout_rate=0.5, l2_lambda=0.01):
    """Creates a simple feedforward neural network with an embedding layer and configurable dense units."""
    
    model = Sequential([
        Embedding(vocab_size, embedding_dim, input_length=input_length),
        Flatten(),  # Flatten the embedding output
        Dense(dense_units, activation='relu', kernel_regularizer=l2(l2_lambda)),  
        Dropout(dropout_rate),  
        Dense(dense_units // 2, activation='relu', kernel_regularizer=l2(l2_lambda)), 
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid', kernel_regularizer=l2(l2_lambda))
    ])

    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model



In [24]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


def train_model(train_data, val_data, epochs=10, batch_size=64, dense_units=64, dropout_rate=0.3, grid_search=False):
    """Trains a simple NN model with word embeddings."""
    seed = 42
    tf.random.set_seed(seed)
    np.random.seed(seed)

    vocab_size = 10000
    max_sequence_length = 300

    print(f"\n🚀 Training Simple NN Model with dense_units = {dense_units}, dropout_rate = {dropout_rate} 🚀\n")

    # Extract texts and labels
    train_texts = train_data["processed_full_content"]
    val_texts = val_data["processed_full_content"]
    y_train = train_data["label"]
    y_val = val_data["label"]

    # Tokenize texts
    tokenizer = Tokenizer(num_words=vocab_size)
    tokenizer.fit_on_texts(train_texts)

    # Convert texts to sequences and pad them
    X_train = pad_sequences(tokenizer.texts_to_sequences(train_texts), maxlen=max_sequence_length)
    X_val = pad_sequences(tokenizer.texts_to_sequences(val_texts), maxlen=max_sequence_length)

    # Create and compile the model
    model = create_basic_nn(vocab_size=vocab_size, input_length=max_sequence_length)

    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Train the model
    model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping],
        verbose=1
    )

     # Evaluate model performance on validation data
    y_pred = (model.predict(X_val) > 0.5).astype(int).reshape(-1)

    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)

    #  Print evaluation metrics
    result = {"accuracy": accuracy, "precision": precision, "recall": recall, "f1_score": f1}
    print("\n🏆 Training Results:")

    for key, value in result.items():
        print(f"🔹 {key.capitalize()}: {value:.4f}")

    return result if grid_search else model

In [25]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_model(model, train_data, val_data):
    """Evaluates the trained NN model."""
    print("📊 Evaluating Model...")

    max_words = 10000
    max_sequence_length = 300

    train_texts = train_data["processed_full_content"]
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(train_texts)

    y_val = val_data["label"]
    val_texts = val_data["processed_full_content"]

    # Convert validation text into padded sequences using the same tokenizer
    X_val = pad_sequences(tokenizer.texts_to_sequences(val_texts), maxlen=max_sequence_length)

    # Predict and threshold
    y_pred = (model.predict(X_val) > 0.5).astype(int).reshape(-1)

    # Compute evaluation metrics
    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1: {f1:.4f}")

    # print("\n🏆 Final Evaluation Results:")
    # for key, value in {"accuracy": accuracy, "precision": precision, "recall": recall, "f1_score": f1}.items():
    #     print(f"🔹 {key.capitalize()}: {value:.4f}")

    # return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1_score": f1}


In [26]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

def do_grid_search(data):
    """Performs Grid Search for the best NN hyperparameters (dropout_rate, dense_units)."""
    
    # ✅ Now testing different dropout rates and dense units
    param_grid = {
        "dropout_rate": [0.3, 0.5],  
        "dense_units": [64, 128]  
    }
    
    results = []
    best_score = 0
    best_params = None

    for dense_units in param_grid["dense_units"]:
        for dropout_rate in param_grid["dropout_rate"]:
            print(f"\n🔍 Testing dense_units={dense_units}, dropout_rate={dropout_rate}")

            kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
            fold_scores = []

            for fold, (train_idx, val_idx) in enumerate(
                kfold.split(data["processed_full_content"], data["label"].to_numpy().ravel()), 1
            ):
                print(f"\n📂 Fold {fold}")

                train_data = data.iloc[train_idx]
                val_data = data.iloc[val_idx]

                # Pass dense_units into train_model
                result = train_model(
                    train_data=train_data,
                    val_data=val_data,
                    dense_units=dense_units,
                    dropout_rate=dropout_rate,
                    grid_search=True
                )
                
                fold_scores.append(result["f1_score"])

            avg_score = np.mean(fold_scores)
            print(f"📊 Average F1-score: {avg_score:.4f}")

            if avg_score > best_score:
                best_score = avg_score
                best_params = {"dense_units": dense_units, "dropout_rate": dropout_rate}

    return best_params


In [27]:
train_data = datasets["train"].to_pandas()
val_data = datasets["test"].to_pandas()
data = pd.concat([train_data, val_data], ignore_index=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60491 entries, 0 to 60490
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   label                   60491 non-null  int64 
 1   full_content            60491 non-null  object
 2   processed_full_content  60491 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [28]:
%%time
best_params = do_grid_search(data)



🔍 Testing dense_units=64, dropout_rate=0.3

📂 Fold 1

🚀 Training Simple NN Model with dense_units = 64, dropout_rate = 0.3 🚀

Epoch 1/10


/home/dariusng2103/projects/dm_project/fake-news/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


757/757 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8616 - loss: 0.7786 - val_accuracy: 0.9688 - val_loss: 0.3252
Epoch 2/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9693 - loss: 0.3503 - val_accuracy: 0.9664 - val_loss: 0.3437
Epoch 3/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9803 - loss: 0.3220 - val_accuracy: 0.9669 - val_loss: 0.3528
Epoch 4/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9844 - loss: 0.3081 - val_accuracy: 0.9666 - val_loss: 0.3282
379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 840us/step

🏆 Training Results:
🔹 Accuracy: 0.9688
🔹 Precision: 0.9598
🔹 Recall: 0.9692
🔹 F1_score: 0.9645

📂 Fold 2

🚀 Training Simple NN Model with dense_units = 64, dropout_rate = 0.3 🚀

Epoch 1/10


/home/dariusng2103/projects/dm_project/fake-news/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


757/757 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8621 - loss: 0.7637 - val_accuracy: 0.9626 - val_loss: 0.3962
Epoch 2/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9701 - loss: 0.3758 - val_accuracy: 0.9640 - val_loss: 0.3875
Epoch 3/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9780 - loss: 0.3341 - val_accuracy: 0.9621 - val_loss: 0.3439
Epoch 4/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9855 - loss: 0.3007 - val_accuracy: 0.9504 - val_loss: 0.4373
Epoch 5/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9868 - loss: 0.3065 - val_accuracy: 0.9536 - val_loss: 0.4000
Epoch 6/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9875 - loss: 0.2845 - val_accuracy: 0.9563 - val_loss: 0.4364
379/379 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

🏆 Training Results:
🔹 Accuracy: 0.9621
🔹 Precision: 0.9693
🔹 Recall: 0.9433
🔹 F1_score: 0.9561

📂 Fold 3

🚀 Training Simple NN Model with dense_units = 64, dropout_rate = 0.3 🚀

Epoch 1/10


...

757/757 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8690 - loss: 0.7724 - val_accuracy: 0.9650 - val_loss: 0.3767
Epoch 2/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9736 - loss: 0.3477 - val_accuracy: 0.9688 - val_loss: 0.3362
Epoch 3/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9820 - loss: 0.3074 - val_accuracy: 0.9678 - val_loss: 0.3509
Epoch 4/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9844 - loss: 0.3034 - val_accuracy: 0.9656 - val_loss: 0.3696
Epoch 5/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9878 - loss: 0.2939 - val_accuracy: 0.9667 - val_loss: 0.3221
Epoch 6/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9871 - loss: 0.2982 - val_accuracy: 0.9602 - val_loss: 0.3840
Epoch 7/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9903 - loss: 0.2683 - val_accuracy: 0.9657 - val_loss: 0.3127
Epoch 8/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9898 - loss: 0.2782 - val_accuracy: 0.9640 - val_

/home/dariusng2103/projects/dm_project/fake-news/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


757/757 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8682 - loss: 0.7814 - val_accuracy: 0.9552 - val_loss: 0.3774
Epoch 2/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9713 - loss: 0.3483 - val_accuracy: 0.9616 - val_loss: 0.3503
Epoch 3/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9792 - loss: 0.3202 - val_accuracy: 0.9634 - val_loss: 0.3563
Epoch 4/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9827 - loss: 0.3177 - val_accuracy: 0.9583 - val_loss: 0.4026
Epoch 5/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9850 - loss: 0.3126 - val_accuracy: 0.9638 - val_loss: 0.3334
Epoch 6/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9874 - loss: 0.2841 - val_accuracy: 0.9621 - val_loss: 0.3553
Epoch 7/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9884 - loss: 0.2947 - val_accuracy: 0.9643 - val_loss: 0.3011
Epoch 8/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9897 - loss: 0.2742 - val_accuracy: 0.9672 - val_

/home/dariusng2103/projects/dm_project/fake-news/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


757/757 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8549 - loss: 0.7663 - val_accuracy: 0.9597 - val_loss: 0.3513
Epoch 2/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9701 - loss: 0.3482 - val_accuracy: 0.9578 - val_loss: 0.3769
Epoch 3/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9804 - loss: 0.3202 - val_accuracy: 0.9551 - val_loss: 0.3755
Epoch 4/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9864 - loss: 0.3003 - val_accuracy: 0.9610 - val_loss: 0.3153
Epoch 5/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9869 - loss: 0.2840 - val_accuracy: 0.9646 - val_loss: 0.3200
Epoch 6/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9879 - loss: 0.2900 - val_accuracy: 0.9594 - val_loss: 0.3775
Epoch 7/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9890 - loss: 0.2859 - val_accuracy: 0.9615 - val_loss: 0.4291
379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 846us/step

🏆 Training Results:
🔹 Accuracy: 0.9610
🔹 Precision: 0.9510
🔹 Recall: 0.

In [29]:
%%time
model = train_model(train_data, val_data, dense_units=best_params["dense_units"], dropout_rate=best_params["dropout_rate"])



🚀 Training Simple NN Model with dense_units = 64, dropout_rate = 0.5 🚀

Epoch 1/10


/home/dariusng2103/projects/dm_project/fake-news/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


851/851 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8594 - loss: 0.7258 - val_accuracy: 0.9608 - val_loss: 0.3820
Epoch 2/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9695 - loss: 0.3525 - val_accuracy: 0.9661 - val_loss: 0.3296
Epoch 3/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9794 - loss: 0.3261 - val_accuracy: 0.9658 - val_loss: 0.3540
Epoch 4/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9836 - loss: 0.3194 - val_accuracy: 0.9612 - val_loss: 0.3732
Epoch 5/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9861 - loss: 0.3159 - val_accuracy: 0.9655 - val_loss: 0.3417
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

🏆 Training Results:
🔹 Accuracy: 0.9661
🔹 Precision: 0.9653
🔹 Recall: 0.9569
🔹 F1_score: 0.9611
CPU times: user 22.5 s, sys: 2.26 s, total: 24.8 s
Wall time: 24.7 s


In [30]:
model.save("results/NN_model_original.keras")

In [31]:
# load model
from tensorflow.keras.models import load_model

model2 = load_model("results/NN_model_original.keras")
model2.summary()

Model: "sequential_41"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_41 (Embedding)        │ (None, 300, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_41 (Flatten)            │ (None, 38400)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_123 (Dense)               │ (None, 64)             │     2,457,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_82 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_124 (Dense)               │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_83 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_125 (Dense)               │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,219,333 (42.80 MB)

 Trainable params: 3,739,777 (14.27 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 7,479,556 (28.53 MB)

In [38]:
%%time
evaluate_model(model, train_data, val_data)

📊 Evaluating Model...
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 0.9661
Precision: 0.9653
Recall: 0.9569
F1: 0.9611
CPU times: user 8.37 s, sys: 38.9 ms, total: 8.41 s
Wall time: 8.31 s


In [39]:
%%time
evaluate_model(model2, train_data, val_data)

📊 Evaluating Model...
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 0.9661
Precision: 0.9653
Recall: 0.9569
F1: 0.9611
CPU times: user 8.28 s, sys: 39.3 ms, total: 8.32 s
Wall time: 8.35 s


In [34]:
val_data_rewritten = datasets["rewritten_test"].to_pandas()
train_data_rewritten = datasets["rewritten_train"].to_pandas()
data_rewritten = pd.concat([train_data, train_data_rewritten, val_data, val_data_rewritten], ignore_index=True)
data_rewritten.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120982 entries, 0 to 120981
Data columns (total 3 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   label                   120982 non-null  int64 
 1   full_content            120982 non-null  object
 2   processed_full_content  120982 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.8+ MB


In [35]:
%%time

evaluate_model(model, train_data, val_data_rewritten)

📊 Evaluating Model...
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 0.8230
Precision: 0.8807
Recall: 0.6887
F1: 0.7729
CPU times: user 8.39 s, sys: 56.3 ms, total: 8.44 s
Wall time: 8.47 s


In [36]:
%%time

evaluate_model(model2, train_data, val_data_rewritten)

📊 Evaluating Model...
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Accuracy: 0.8230
Precision: 0.8807
Recall: 0.6887
F1: 0.7729
CPU times: user 8.32 s, sys: 4.32 ms, total: 8.32 s
Wall time: 8.35 s


In [37]:
%%time

best_params_rewritten = do_grid_search(data_rewritten)
best_params_rewritten


🔍 Testing dense_units=64, dropout_rate=0.3

📂 Fold 1

🚀 Training Simple NN Model with dense_units = 64, dropout_rate = 0.3 🚀

Epoch 1/10


/home/dariusng2103/projects/dm_project/fake-news/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1513/1513 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8442 - loss: 0.7573 - val_accuracy: 0.9090 - val_loss: 0.4727
Epoch 2/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9355 - loss: 0.4710 - val_accuracy: 0.9319 - val_loss: 0.4438
Epoch 3/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9508 - loss: 0.4812 - val_accuracy: 0.9226 - val_loss: 0.5747
Epoch 4/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9626 - loss: 0.4818 - val_accuracy: 0.9299 - val_loss: 0.5354
Epoch 5/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9677 - loss: 0.4858 - val_accuracy: 0.9332 - val_loss: 0.5583
757/757 ━━━━━━━━━━━━━━━━━━━━ 1s 754us/step

🏆 Training Results:
🔹 Accuracy: 0.9319
🔹 Precision: 0.9424
🔹 Recall: 0.8992
🔹 F1_score: 0.9203

📂 Fold 2

🚀 Training Simple NN Model with dense_units = 64, dropout_rate = 0.3 🚀

Epoch 1/10


/home/dariusng2103/projects/dm_project/fake-news/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1513/1513 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8454 - loss: 0.7193 - val_accuracy: 0.9218 - val_loss: 0.4664
Epoch 2/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9354 - loss: 0.4657 - val_accuracy: 0.9212 - val_loss: 0.4900
Epoch 3/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9482 - loss: 0.4713 - val_accuracy: 0.9198 - val_loss: 0.5197
Epoch 4/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9589 - loss: 0.4797 - val_accuracy: 0.9208 - val_loss: 0.5366
757/757 ━━━━━━━━━━━━━━━━━━━━ 1s 758us/step

🏆 Training Results:
🔹 Accuracy: 0.9218
🔹 Precision: 0.9287
🔹 Recall: 0.8896
🔹 F1_score: 0.9087

📂 Fold 3

🚀 Training Simple NN Model with dense_units = 64, dropout_rate = 0.3 🚀

Epoch 1/10


...

/home/dariusng2103/projects/dm_project/fake-news/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1513/1513 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8357 - loss: 0.7287 - val_accuracy: 0.9253 - val_loss: 0.4646
Epoch 2/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9355 - loss: 0.4651 - val_accuracy: 0.9266 - val_loss: 0.4763
Epoch 3/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9491 - loss: 0.4760 - val_accuracy: 0.9261 - val_loss: 0.4967
Epoch 4/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9587 - loss: 0.4699 - val_accuracy: 0.9244 - val_loss: 0.5226
757/757 ━━━━━━━━━━━━━━━━━━━━ 1s 739us/step

🏆 Training Results:
🔹 Accuracy: 0.9253
🔹 Precision: 0.9178
🔹 Recall: 0.9109
🔹 F1_score: 0.9143

📂 Fold 5

🚀 Training Simple NN Model with dense_units = 128, dropout_rate = 0.5 🚀

Epoch 1/10


/home/dariusng2103/projects/dm_project/fake-news/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1513/1513 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8438 - loss: 0.7329 - val_accuracy: 0.9264 - val_loss: 0.4500
Epoch 2/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9372 - loss: 0.4794 - val_accuracy: 0.9268 - val_loss: 0.5028
Epoch 3/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9504 - loss: 0.4772 - val_accuracy: 0.9168 - val_loss: 0.5413
Epoch 4/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9589 - loss: 0.4915 - val_accuracy: 0.9244 - val_loss: 0.5910
757/757 ━━━━━━━━━━━━━━━━━━━━ 1s 739us/step

🏆 Training Results:
🔹 Accuracy: 0.9263
🔹 Precision: 0.9070
🔹 Recall: 0.9266
🔹 F1_score: 0.9167
📊 Average F1-score: 0.9152
CPU times: user 11min 43s, sys: 1min 6s, total: 12min 50s
Wall time: 12min 52s


{'dense_units': 128, 'dropout_rate': 0.5}

In [41]:
%%time

train_data_combined = pd.concat([train_data, train_data_rewritten], ignore_index=True)
val_data_combined = pd.concat([val_data, val_data_rewritten], ignore_index=True)
model_combined = train_model(train_data_combined, val_data_combined, 
                             dense_units=best_params_rewritten["dense_units"], 
                             dropout_rate=best_params_rewritten["dropout_rate"])
model_combined.save("results/NN_model_combined.keras")


🚀 Training Simple NN Model with dense_units = 128, dropout_rate = 0.5 🚀

Epoch 1/10


/home/dariusng2103/projects/dm_project/fake-news/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1702/1702 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8505 - loss: 0.7173 - val_accuracy: 0.9288 - val_loss: 0.4288
Epoch 2/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9352 - loss: 0.4688 - val_accuracy: 0.9269 - val_loss: 0.4771
Epoch 3/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9493 - loss: 0.4785 - val_accuracy: 0.9250 - val_loss: 0.5300
Epoch 4/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9584 - loss: 0.4858 - val_accuracy: 0.9217 - val_loss: 0.5693
379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 895us/step

🏆 Training Results:
🔹 Accuracy: 0.9288
🔹 Precision: 0.9172
🔹 Recall: 0.9203
🔹 F1_score: 0.9187
CPU times: user 37.8 s, sys: 3.45 s, total: 41.3 s
Wall time: 41.6 s


In [42]:
evaluate_model(model_combined, train_data_combined, val_data_combined)

📊 Evaluating Model...
379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 0.9288
Precision: 0.9172
Recall: 0.9203
F1: 0.9187


In [43]:
evaluate_model(model_combined, train_data_combined, val_data)

📊 Evaluating Model...
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Accuracy: 0.9597
Precision: 0.9570
Recall: 0.9505
F1: 0.9538


In [44]:
evaluate_model(model_combined, train_data_combined, val_data_rewritten)

📊 Evaluating Model...
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 0.8979
Precision: 0.8781
Recall: 0.8901
F1: 0.8841


In [45]:
model_combined2 = load_model("results/NN_model_combined.keras")
model_combined2.summary()

Model: "sequential_62"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_62 (Embedding)        │ (None, 300, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_62 (Flatten)            │ (None, 38400)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_186 (Dense)               │ (None, 64)             │     2,457,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_124 (Dropout)           │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_187 (Dense)               │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_125 (Dropout)           │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_188 (Dense)               │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,219,333 (42.80 MB)

 Trainable params: 3,739,777 (14.27 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 7,479,556 (28.53 MB)

In [46]:
evaluate_model(model_combined2, train_data_combined, val_data_combined)

📊 Evaluating Model...
379/379 ━━━━━━━━━━━━━━━━━━━━ 0s 929us/step
Accuracy: 0.9288
Precision: 0.9172
Recall: 0.9203
F1: 0.9187


In [47]:
evaluate_model(model_combined2, train_data_combined, val_data)

📊 Evaluating Model...
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
Accuracy: 0.9597
Precision: 0.9570
Recall: 0.9505
F1: 0.9538


In [48]:
evaluate_model(model_combined2, train_data_combined, val_data_rewritten)

📊 Evaluating Model...
190/190 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Accuracy: 0.8979
Precision: 0.8781
Recall: 0.8901
F1: 0.8841
